<a href="https://colab.research.google.com/github/Veritate1311/Portfolio/blob/Datascience_algo/Yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --quiet ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.1/800.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 14.2 MB/s eta 0:00:00


In [2]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
train_path_img = "/content/drive/MyDrive/Yolov8/training/images/"
train_path_label = "/content/drive/MyDrive/Yolov8/training/labels/"
val_path_img = "/content/drive/MyDrive/Yolov8/testing/images/"
val_path_label = "/content/drive/MyDrive/Yolov8/testing/labels/"
test_path = "/content/drive/MyDrive/Yolov8/testing"


In [9]:
### for label_tag
train_test_split('/content/drive/MyDrive/Yolov8/Data') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 66 images---


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split 53 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

In [15]:
import os
import shutil
import random
from tqdm.notebook import tqdm

# Paths
base_path = "/content/drive/MyDrive/Yolov8"
data_path = base_path  # Using the base path directly for data
train_path_img = os.path.join(base_path, "training/images")
train_path_label = os.path.join(base_path, "training/labels")
val_path_img = os.path.join(base_path, "testing/images")
val_path_label = os.path.join(base_path, "testing/labels")

def train_val_split(path, val_split=0.2):
    print("------ PROCESS STARTED -------")

    # Check if the provided path exists
    if not os.path.exists(path):
        print(f"Error: The specified path '{path}' does not exist.")
        return

    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')])) ## removing duplicate names i.e. counting only number of images

    print(f"--- This folder has a total number of {len(files)} images ---")
    random.seed(42)
    random.shuffle(files)

    val_size = int(len(files) * val_split)
    train_size = len(files) - val_size

    # Creating required directories
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    # Copying images and labels to train folder
    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        # Check if the image and label files exist before copying
        image_path = os.path.join(path, filex + '.jpg')
        label_path = os.path.join(path, filex + '.txt')
        if os.path.exists(image_path):
            shutil.copy2(image_path, os.path.join(train_path_img, filex + '.jpg'))
        else:
            print(f"Warning: Image file not found: {image_path}")
        if os.path.exists(label_path):
            shutil.copy2(label_path, os.path.join(train_path_label, filex + '.txt'))
        else:
            print(f"Warning: Label file not found: {label_path}")

    print(f"------ Training data created with {100 - val_split*100:.0f}% split ({len(files[:train_size])} images) -------")

    # Copying images and labels to validation folder
    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        # Check if the image and label files exist before copying
        image_path = os.path.join(path, filex + '.jpg')
        label_path = os.path.join(path, filex + '.txt')
        if os.path.exists(image_path):
            shutil.copy2(image_path, os.path.join(val_path_img, filex + '.jpg'))
        else:
            print(f"Warning: Image file not found: {image_path}")
        if os.path.exists(label_path):
            shutil.copy2(label_path, os.path.join(val_path_label, filex + '.txt'))
        else:
            print(f"Warning: Label file not found: {label_path}")

    print(f"------ Validation data created with {val_split*100:.0f}% split ({len(files[train_size:])} images) -------")

# Run the function to split data into training and validation sets
train_val_split(data_path)

# Install ultralytics and check setup
!pip install ultralytics
import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.2/78.2 GB disk)


In [17]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/Yolov8/training name=football


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Yolov8/dataset.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Yolov8/training, name=football5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sav

In [18]:
import os
import shutil
import random
from tqdm.notebook import tqdm

# Paths
base_path = "/content/drive/MyDrive/Yolov8"
data_path = os.path.join(base_path, "Data")  # Using Data directory for input images
train_path_img = os.path.join(base_path, "Data/train/images")
train_path_label = os.path.join(base_path, "Data/train/labels")
val_path_img = os.path.join(base_path, "Data/val/images")
val_path_label = os.path.join(base_path, "Data/val/labels")

def train_val_split(path, val_split=0.2):
    print("------ PROCESS STARTED -------")

    # Check if the provided path exists
    if not os.path.exists(path):
        print(f"Error: The specified path '{path}' does not exist.")
        return

    files = list(set([name[:-4] for name in os.listdir(path) if name.endswith('.jpg')])) ## removing duplicate names i.e. counting only number of images

    print(f"--- This folder has a total number of {len(files)} images ---")
    random.seed(42)
    random.shuffle(files)

    val_size = int(len(files) * val_split)
    train_size = len(files) - val_size

    # Creating required directories
    os.makedirs(train_path_img, exist_ok=True)
    os.makedirs(train_path_label, exist_ok=True)
    os.makedirs(val_path_img, exist_ok=True)
    os.makedirs(val_path_label, exist_ok=True)

    # Copying images and labels to train folder
    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        # Check if the image and label files exist before copying
        image_path = os.path.join(path, filex + '.jpg')
        label_path = os.path.join(path, filex + '.txt')
        if os.path.exists(image_path):
            shutil.copy2(image_path, os.path.join(train_path_img, filex + '.jpg'))
        else:
            print(f"Warning: Image file not found: {image_path}")
        if os.path.exists(label_path):
            shutil.copy2(label_path, os.path.join(train_path_label, filex + '.txt'))
        else:
            print(f"Warning: Label file not found: {label_path}")

    print(f"------ Training data created with {100 - val_split*100:.0f}% split ({len(files[:train_size])} images) -------")

    # Copying images and labels to validation folder
    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        # Check if the image and label files exist before copying
        image_path = os.path.join(path, filex + '.jpg')
        label_path = os.path.join(path, filex + '.txt')
        if os.path.exists(image_path):
            shutil.copy2(image_path, os.path.join(val_path_img, filex + '.jpg'))
        else:
            print(f"Warning: Image file not found: {image_path}")
        if os.path.exists(label_path):
            shutil.copy2(label_path, os.path.join(val_path_label, filex + '.txt'))
        else:
            print(f"Warning: Label file not found: {label_path}")

    print(f"------ Validation data created with {val_split*100:.0f}% split ({len(files[train_size:])} images) -------")

# Run the function to split data into training and validation sets
train_val_split(data_path)

# Install ultralytics and check setup
!pip install ultralytics
import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.3/78.2 GB disk)


In [19]:
!ls /content/drive/MyDrive/Yolov8/Data/train/images
!ls /content/drive/MyDrive/Yolov8/Data/train/labels
!ls /content/drive/MyDrive/Yolov8/Data/val/images
!ls /content/drive/MyDrive/Yolov8/Data/val/labels



22d_f_125_fre.jpg  22d_f_85_fr.jpg    6ty_f_80_54r.jpg	 fre_f_60.jpg	   rt3_f_170_6ty.jpg
22d_f_125.jpg	   22d_f_90.jpg       6ty_f_85.jpg	 fre_f_95_54r.jpg  rt3_f_20_54r.jpg
22d_f_135_we.jpg   54r_f_120.jpg      fre_f_10_54r.jpg	 fr_f_105_6ty.jpg  rt3_f_95.jpg
22d_f_165_fr.jpg   54r_f_65.jpg       fre_f_115.jpg	 fr_f_10.jpg	   we_f_100.jpg
22d_f_180_6ty.jpg  54r_f_75.jpg       fre_f_145_6ty.jpg  fr_f_15.jpg	   we_f_115_fre.jpg
22d_f_185_6ty.jpg  54r_f_75_we.jpg    fre_f_155_we.jpg	 fr_f_160_fre.jpg  we_f_150_we.jpg
22d_f_200_rt3.jpg  6ty_f_110_54r.jpg  fre_f_190_22d.jpg  fr_f_40.jpg	   we_f_20.jpg
22d_f_25.jpg	   6ty_f_130_22d.jpg  fre_f_35.jpg	 fr_f_45.jpg	   we_f_30.jpg
22d_f_30_22d.jpg   6ty_f_195_6ty.jpg  fre_f_40_22d.jpg	 fr_f_55.jpg	   we_f_50.jpg
22d_f_55_22d.jpg   6ty_f_50_we.jpg    fre_f_5.jpg	 fr_f_70.jpg
22d_f_80.jpg	   6ty_f_65_we.jpg    fre_f_60_fr.jpg	 rt3_f_110.jpg
22d_f_125_fre.txt  22d_f_85_fr.txt    6ty_f_80_54r.txt	 fre_f_60.txt	   rt3_f_170_6ty.txt
22d_f_125.txt	  

In [21]:
# Train the model
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/Yolov8/Data/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/Yolov8/training name=football


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/Yolov8/Data/dataset.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Yolov8/training, name=football7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False

In [25]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Yolov8/training/football7/weights/best.pt conf=0.55 source=/content/drive/MyDrive/Yolov8/testing


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/5 /content/drive/MyDrive/Yolov8/testing/54r_f_120.jpg: 384x640 5 players, 124.4ms
image 2/5 /content/drive/MyDrive/Yolov8/testing/54r_f_75.jpg: 384x640 7 players, 12.8ms
image 3/5 /content/drive/MyDrive/Yolov8/testing/we_f_105 (1).jpg: 384x640 9 players, 10.8ms
image 4/5 /content/drive/MyDrive/Yolov8/testing/we_f_105.jpg: 384x640 9 players, 10.8ms
image 5/5 /content/drive/MyDrive/Yolov8/testing/we_f_175_we.jpg: 384x640 15 players, 10.8ms
Speed: 3.0ms preprocess, 33.9ms inference, 184.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [27]:
!cp -r /content/runs/detect/predict //content/drive/MyDrive/Yolov8/output

In [28]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/Yolov8/training/football7/weights/best.pt conf=0.77 source=/content/drive/MyDrive/Yolov8/vid


Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/6 (frame 1/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps.mp4: 640x352 (no detections), 132.7ms
video 1/6 (frame 2/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps.mp4: 640x352 (no detections), 11.1ms
video 1/6 (frame 3/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps.mp4: 640x352 (no detections), 11.5ms
video 1/6 (frame 4/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps.mp4: 640x352 (no detections), 11.4ms
video 1/6 (frame 5/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps.mp4: 640x352 (no detections), 19.4ms
video 1/6 (frame 6/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps.mp4: 640x352 (no detections), 11.2ms
video 1/6 (frame 7/157) /content/drive/MyDrive/Yolov8/vid/10349008-uhd_2160_4096_25fps

In [29]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/Yolov8/output